This is an attempt to try mimic the simplified F3 workflow in Python. Some details are missing as the F3 design is not fully represented and the successful run of this Python notebook does not guarantee a smooth implementation in F3.

In [1040]:
import math
from scipy.stats import norm
import scipy.interpolate
import numpy as np
import copy
import bisect

In [1041]:
### Helper function

### Take in expiry time, ratio strike, and volatility, returns the normalized European call option price and its vega
class BlackPricerInF3:
    def __init__(self, expiryTime, ratioStrike, volatility):
        vol_sqrt_t = volatility*math.sqrt(expiryTime)
        self.m_D1 = (math.log(1./ratioStrike) + 0.5*vol_sqrt_t*vol_sqrt_t) / vol_sqrt_t
        self.m_D2 = self.m_D1 - vol_sqrt_t
        self.m_K = ratioStrike
        self.m_T = expiryTime
    def price(self):
        return norm.cdf(self.m_D1) - self.m_K*norm.cdf(self.m_D2)
    def vega(self):
        return norm.pdf(self.m_D1)*math.sqrt(self.m_T)

In [1042]:
### Helper function

### Form a piece-wise constant volatility function given the market ratio strikes and the volatilities, vol(k),
### This is a function of the ratio strike
### The mid-point of adjacent market ratio strikes is used to mark each segment
### F3 naming
def piecewiseConstantCenterAndSnapLeftOnCenterPointInterpolationBuilder(marketRatioStrikes, volatilities):
    segmentPoints = []
    for i in range(0, len(marketRatioStrikes) - 1):
        segmentPoints.append((marketRatioStrikes[i+1] + marketRatioStrikes[i])/2.)
    return scipy.interpolate.interp1d(x = segmentPoints,
                                      y = volatilities[0:len(segmentPoints)],
                                      kind = 'next',
                                      bounds_error = False,
                                      fill_value = (volatilities[0], volatilities[-1]))

In [1043]:
### Helper function

### Solution of tri-diagonal system
### A*x = rhs, where A is a tri-diagonal matrix
### Thomas algorithm
### a - subdiagonal, b - diagonal, c - superdiagonal, x - solution, r - righ hand side (solution after the sucessful run)
def solveTridiagonalSystem(a, b, c, r):
    size = len(r)
    # 1, Use linear combinations of rows to eliminate subdiagonal and make main diagonal equal to 1s
    for i in range(0,size-1):
        c[i] = c[i]/b[i]
        r[i] = r[i]/b[i]
        b[i+1] = b[i+1] - c[i]*a[i]
        r[i+1] = r[i+1] - r[i]*a[i]
    r[-1] = r[-1]/b[-1]
    # 2, back-substitute, eliminating the cs, after this loop, x = r
    for i in range(size-2, -1, -1):
        r[i] -= r[i+1]*c[i]

In [1044]:
class EuropeanCallOptionPriceCurvePricer:
    def __init__(self, callPrice3dFunction):
        self.m_CallFunc = callPrice3dFunction
    def optionValue(self, strike, forward, time):
        return self.m_CallFunc.value(strike, forward, time)

In [1045]:
class Model:
    def __init__(self, expiries, ratioStrikes, quotes):
        self.m_Expiries = expiries
        self.m_Strikes = ratioStrikes
        self.m_Quotes = quotes
        self.m_CurveMap = {}
        self.m_CurveCalibrationStatus = {}
    def addOrUpdateZeroDimCurveCalibration(self, tag, calibratedVols):
        self.m_CurveMap[tag] = calibratedVols
    def setCalibrationStatus(self, tag, flag):
        self.m_CurveCalibrationStatus[tag] = flag # 0 not calibrated, 1 Calibrating, 2 Calibrated
    def curveIsCalibrated(self, tag):
        return self.m_CurveCalibrationStatus[tag] == 2
    def curveIsCalibrating(self, tag):
        return self.m_CurveCalibrationStatus[tag] == 1
    def expiry(self, tag):
        return self.m_Expiries[tag]
    def strikes(self, tag):
        return self.m_Strikes[tag]
    def marketData(self, tag):
        return self.m_Quotes[tag]
    def calibratedVols(self, tag):
        return self.m_CurveMap[tag]
    def extendModelWithSuchCurve(self, pricer):
        self.m_CallPricer = pricer
        return self
    def likelihoodFunc(self, vols, tag):
        t = self.m_Expiries[tag]
        self.addOrUpdateZeroDimCurveCalibration(tag, vols)
        residuals = []
        for i in range(len(self.m_Strikes[tag])):
            k = self.m_Strikes[tag][i]
            blackPricer = BlackPricerInF3(t, k, self.m_Quotes[tag][i])
            modelValue = (self.m_CallPricer.optionValue(k, 1., t))
            mdValue = (blackPricer.price())
            residuals.append(abs((modelValue - mdValue)/blackPricer.vega()))
        return residuals
    def valueUseCalibratedPricer(self, k, f, t):
        return self.m_CallPricer.optionValue(k, f, t)
    def getCurve(self, tag):
        if self.curveIsCalibrated(tag) or self.curveIsCalibrating(tag):
            return self.calibratedVols(tag)
        else:
            self.setCalibrationStatus(tag, 1)
            calibrationStart = time.time()
            scipy.optimize.least_squares(self.likelihoodFunc,
                                         self.marketData(tag),
                                         method='lm',
                                         args = ([tag,]),
                                         xtol = 1e-5,
                                         gtol = 1e-5,
                                         ftol = 1e-5)
            calibrationEnd = time.time()
            self.setCalibrationStatus(tag, 2)
            print (f"calibration {tag} finishes in {calibrationEnd - calibrationStart} seconds")
            return self.calibratedVols(tag)

In [1046]:
class PiecewiseConstantVolatility:
    def __init__(self, tag, model):
        self.m_Tag = tag
        self.m_Model = model
    def value(self, k):
        calibratedParams = self.m_Model.getCurve(self.m_Tag)
        pwcFunc = piecewiseConstantCenterAndSnapLeftOnCenterPointInterpolationBuilder(self.m_Model.strikes(self.m_Tag), calibratedParams)
        return pwcFunc(k)
    def underlyingIsCalibrated(self):
        return self.m_Model.curveIsCalibrated(self.m_Tag)
        

In [1047]:
class ArbitrageFreeInterpolationEuroCallOptionPrice:
    def __init__(self, timeVolFuncMap, minY, maxY, size):
        self.m_TimeVolFuncMap = timeVolFuncMap
        self.m_MinY = minY
        self.m_MaxY = maxY
        self.m_Size = size
        self.m_Dy = (self.m_MaxY - self.m_MinY)/self.m_Size
        self.m_Kgrid = [math.exp(self.m_MinY + i*self.m_Dy) for i in range(self.m_Size)]
        self.m_CachedTimePriceMap = {}
        mdExpTimes = self.m_TimeVolFuncMap.keys()
        self.m_MdExpTimes = [mdT for mdT in mdExpTimes]
        self.m_IntrinsicPrices = [max(1-strikeGridElem, 0.) for strikeGridElem in self.m_Kgrid]
    def valueFuncAt(self, t):

        if (t > self.m_MdExpTimes[-1]):
            i = len(self.m_MdExpTimes)-1
        else:
            i = bisect.bisect_left(self.m_MdExpTimes, t)
        previousCallPrices = []
        dt = t
        if i == 0:
            previousCallPrices = copy.deepcopy(self.m_IntrinsicPrices)
        else:
            prevTime = self.m_MdExpTimes[i-1]
            if (prevTime in self.m_CachedTimePriceMap):
                previousCallPrices = copy.deepcopy(self.m_CachedTimePriceMap[prevTime])
            else:
                func = self.valueFuncAt(prevTime) ### Recursion
                previousCallPrices = [func(k) for k in self.m_Kgrid]
            dt = t - prevTime

        volFunc = self.m_TimeVolFuncMap[self.m_MdExpTimes[i]]
        calibVolatilityForGridStrikes = [volFunc.value(k) for k in self.m_Kgrid] ### Trigger the calibration
        ### form tri-diagonal system
        z = [dt*vol*vol/(2.*self.m_Dy) for vol in calibVolatilityForGridStrikes]
        subdiag = []
        diag = []
        superdiag = []
        diag.append(1.)
        superdiag.append(0.)
        for j in range(1, len(self.m_Kgrid)-1):
            diag.append(1.+2.*z[j]/self.m_Dy)
            subdiag.append(-z[j]*(1./self.m_Dy + 0.5))
            superdiag.append(-z[j]*(1./self.m_Dy - 0.5))
        diag.append(1.)
        subdiag.append(0.)
        ### solve tri-diagonal system
        solveTridiagonalSystem(subdiag, diag, superdiag, previousCallPrices)
        interpPriceFunction_t = scipy.interpolate.interp1d(x = self.m_Kgrid,
                                                           y = previousCallPrices)
        if volFunc.underlyingIsCalibrated():
            self.m_CachedTimePriceMap[self.m_MdExpTimes[i]] = previousCallPrices
        return interpPriceFunction_t
    def value(self, k, f, t):
        func = self.valueFuncAt(t)
        return f*func(k/f)
            

In [1048]:
### set up market data for calibration
expTimes = [0.025, 0.101, 0.197, 0.274, 0.523, 0.772, 1.769, 2.267, 2.784, 3.781, 4.778, 5.774]
ratioStrikesMD = [[0.8613, 0.8796, 0.8979, 0.9163, 0.9346, 0.9529, 0.9712, 0.9896, 1.0079, 1.0262, 1.0445, 1.0629, 1.0812, 1.0995, 1.1178],
                [0.8796, 0.8979, 0.9163, 0.9346, 0.9529, 0.9712, 0.9896, 1.0079, 1.0262, 1.0445, 1.0629, 1.0812, 1.0995, 1.1178],
                [0.8796, 0.8979, 0.9163, 0.9346, 0.9529, 0.9712, 0.9896, 1.0079, 1.0262, 1.0445, 1.0629, 1.0812, 1.0995, 1.1178],
                [0.7697, 0.8063, 0.8430, 0.8796, 0.9163, 0.9529, 0.9896, 1.0262, 1.0629, 1.0995, 1.1362, 1.1728, 1.2095, 1.2461],
                [0.7697, 0.8063, 0.8430, 0.8796, 0.9163, 0.9529, 0.9896, 1.0262, 1.0629, 1.0995, 1.1362, 1.1728, 1.2095, 1.2461],
                [0.7697, 0.8063, 0.8430, 0.8796, 0.9163, 0.9529, 0.9896, 1.0262, 1.0629, 1.0995, 1.1362, 1.1728, 1.2095, 1.2461],
                [0.7697, 0.8063, 0.8430, 0.8796, 0.9163, 0.9529, 0.9896, 1.0262, 1.0629, 1.0995, 1.1362, 1.1728, 1.2095, 1.2461],
                [0.8063, 0.8796, 0.9529, 1.0262, 1.0995, 1.1728],
                [0.5131, 0.5864, 0.6597, 0.7330, 0.8063, 0.8796, 0.9529, 1.0262, 1.0995, 1.1728, 1.2461, 1.3194, 1.3927, 1.4660],
                [0.5131, 0.5864, 0.6597, 0.7330, 0.8063, 0.8796, 1.0262, 1.0995, 1.1728, 1.2461, 1.3194, 1.3927, 1.4660],
                [0.6597, 0.7330, 0.8063, 0.8796, 0.9529, 1.0262, 1.0995, 1.1728, 1.2461, 1.3194, 1.3927, 1.4660],
                [0.8063, 0.8796, 0.9529, 1.0262, 1.0995, 1.1728, 1.2461, 1.3194, 1.3927]]
volsMD = [[0.3365, 0.3216, 0.3043, 0.2880, 0.2724, 0.2586, 0.2466, 0.2358, 0.2247, 0.2159, 0.2091, 0.2056, 0.2045, 0.2025, 0.1933],
        [0.2906, 0.2797, 0.2690, 0.2590, 0.2488, 0.2390, 0.2300, 0.2213, 0.2140, 0.2076, 0.2024, 0.1982, 0.1959, 0.1929],
        [0.2764, 0.2672, 0.2578, 0.2489, 0.2405, 0.2329, 0.2253, 0.2184, 0.2123, 0.2069, 0.2025, 0.1984, 0.1944, 0.1920],
        [0.3262, 0.3058, 0.2887, 0.2717, 0.2557, 0.2407, 0.2269, 0.2142, 0.2039, 0.1962, 0.1902, 0.1885, 0.1867, 0.1871],
        [0.3079, 0.2936, 0.2798, 0.2663, 0.2531, 0.2404, 0.2284, 0.2173, 0.2074, 0.1988, 0.1914, 0.1854, 0.1811, 0.1785],
        [0.3001, 0.2876, 0.2750, 0.2637, 0.2519, 0.2411, 0.2299, 0.2198, 0.2104, 0.2022, 0.1950, 0.1888, 0.1839, 0.1793],
        [0.2843, 0.2753, 0.2666, 0.2575, 0.2497, 0.2418, 0.2347, 0.2283, 0.2213, 0.2151, 0.2091, 0.2039, 0.1990, 0.1945],
        [0.2713, 0.2555, 0.2410, 0.2275, 0.2161, 0.2058],
        [0.3366, 0.3178, 0.3019, 0.2863, 0.2711, 0.2580, 0.2448, 0.2322, 0.2219, 0.2122, 0.2054, 0.1988, 0.1930, 0.1849],
        [0.3291, 0.3129, 0.2976, 0.2848, 0.2711, 0.2585, 0.2384, 0.2269, 0.2186, 0.2103, 0.2054, 0.2002, 0.1964],
        [0.2975, 0.2848, 0.2722, 0.2611, 0.2501, 0.2392, 0.2305, 0.2223, 0.2164, 0.2105, 0.2054, 0.2012],
        [0.2809, 0.2693, 0.2584, 0.2486, 0.2399, 0.2321, 0.2251, 0.2190, 0.2135]]

In [1049]:
model = Model(expTimes, ratioStrikesMD, volsMD)
numberOfCalib = len(volsMD)
volFuncMap = {}
for i in range(0, numberOfCalib):
    volFuncMap[expTimes[i]] = PiecewiseConstantVolatility(i, model)
    model.setCalibrationStatus(i, 0)
threeDFunc = ArbitrageFreeInterpolationEuroCallOptionPrice(volFuncMap, math.log(0.3), math.log(1.9), 100)
pricer = EuropeanCallOptionPriceCurvePricer(threeDFunc)
model.extendModelWithSuchCurve(pricer)


In [1050]:
import time

start = time.time()
print (model.valueUseCalibratedPricer(0.9712, 1., 5.774))
end = time.time()
print(f"time cost: {end - start}")

calibration 0 finishes in 14.277796983718872 seconds
calibration 1 finishes in 7.638868570327759 seconds
calibration 2 finishes in 7.654489994049072 seconds
calibration 3 finishes in 7.65448784828186 seconds
calibration 4 finishes in 7.607619047164917 seconds
calibration 5 finishes in 7.63886833190918 seconds
calibration 6 finishes in 8.810477018356323 seconds
calibration 7 finishes in 1.5465185642242432 seconds
calibration 8 finishes in 8.826123237609863 seconds
calibration 9 finishes in 7.654500246047974 seconds
calibration 10 finishes in 6.560986042022705 seconds
calibration 11 finishes in 3.241579294204712 seconds
0.25263776694146695
time cost: 89.2216362953186


In [1051]:
### Roundtripping

In [1052]:
priceErrors = []
for time, strikes, vols in zip(expTimes, ratioStrikesMD, volsMD):
    errors = []
    for k, v in zip(strikes, vols):
        marketPricer = BlackPricerInF3(time, k, v)
        modelPrice = model.valueUseCalibratedPricer(k, 1., time)
        errors.append(abs(modelPrice - marketPricer.price()))
    priceErrors.append(errors)

In [1053]:
print (priceErrors)

[[8.83182416089312e-14, 3.358147093734942e-13, 7.270008761661728e-10, 1.228652711515732e-08, 7.631359422166639e-08, 3.4839200861030983e-07, 1.3879971603944496e-06, 4.3039778894089775e-06, 8.516159626076325e-06, 8.922332177601147e-06, 4.273817421711866e-06, 9.023934514758802e-07, 8.848311738206749e-08, 3.20783910320036e-09, 2.832897582828131e-10], [3.454353053600201e-10, 2.2186562309567393e-08, 1.3951065226669446e-07, 4.075625861660326e-07, 8.859240927250323e-07, 1.9301914066655046e-06, 4.18766961238215e-06, 8.11867829035584e-06, 1.2587467537003572e-05, 1.4397972408132509e-05, 1.1396226943704603e-05, 5.973846889542698e-06, 1.8713749788735502e-06, 3.21892086916015e-07], [1.386903203393075e-09, 8.743367974073379e-10, 1.7575780927869822e-08, 3.507882828657216e-08, 7.866640955311865e-08, 1.3949743044722185e-07, 3.0759167248678443e-07, 5.642119666748879e-07, 9.712248267498835e-07, 1.2290279113487401e-06, 1.2885398031920597e-06, 8.329374630803438e-07, 4.895974156119906e-07, 3.198605374142571e

In [1054]:
max(priceErrors)

[5.32095990046777e-08,
 6.468715488261445e-08,
 7.861066647629755e-08,
 9.465569275612928e-08,
 1.1551126349873542e-07,
 1.4010330262603787e-07,
 1.71148126365539e-07,
 2.0960100363676126e-07,
 2.5449832742607814e-07,
 3.1157034648121584e-07,
 3.822233755959581e-07,
 4.7316379629680494e-07,
 5.885482663448016e-07,
 7.277768835980969e-07]